In [79]:
from langgraph.graph import START,END,StateGraph
from typing import TypedDict ,Literal,Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
import operator

In [80]:
generator_llm = ChatOpenAI(model="gpt-4o-mini")   # fast + cheap for generation
evaluator_llm = ChatOpenAI(model="gpt-4o")       # strong reasoning for evaluation
optimizer_llm = ChatOpenAI(model="gpt-4.1")      # deep optimization/refinement


In [81]:
from pydantic import BaseModel, Field

class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")

In [82]:
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [83]:
# defining State

class TweetState(TypedDict):

    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int

    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [84]:
def generate_tweet(state: TweetState):

    # prompt
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]

    # send generator_llm
    response = generator_llm.invoke(messages).content

    # return response
    return {'tweet': response, 'tweet_history': [response]}

In [85]:
def evaluate_tweet(state: TweetState):

    # prompt
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")
]

    response = structured_evaluator_llm.invoke(messages)

    return {'evaluation':response.evaluation, 'feedback': response.feedback, 'feedback_history': [response.feedback]}

In [86]:
def optimize_tweet(state: TweetState):

    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1

    return {'tweet': response, 'iteration': iteration, 'tweet_history': [response]}

In [87]:
#condition
def route_evaluation(state: TweetState):

    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

In [88]:
#setting graph
graph=StateGraph(TweetState)
#nodes
graph.add_node('X_Æ-01',generate_tweet)
graph.add_node('X_Æ-02',evaluate_tweet)
graph.add_node('X_Æ-03',optimize_tweet)

#edges
graph.add_edge(START,'X_Æ-01')
graph.add_edge('X_Æ-01','X_Æ-02')
graph.add_conditional_edges('X_Æ-02', route_evaluation, {'approved': END, 'needs_improvement': 'X_Æ-03'})
graph.add_edge('X_Æ-03','X_Æ-02')

#compile graph
workflow=graph.compile()




In [89]:
#show workflow
workflow


ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 404.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [95]:
initial_state = {
    "topic": "openai",
    "iteration": 1,
    "max_iteration": 5
}
result = workflow.invoke(initial_state)

In [96]:
result

{'topic': 'openai',
 'tweet': "OpenAI is like that friend who’s always reading the room but still decides to show up in a dinosaur costume. Sure, it's impressive, but you gotta wonder if they really know we're just here for pizza. 🦖🍕 #AIInFashion #WhoInvitedThisBot",
 'evaluation': 'approved',
 'feedback': "The tweet is original, using a creative analogy comparing OpenAI to a quirky friend in a dinosaur costume, which is an uncommon and vivid metaphor that captures attention. It carries humor through the ridiculous juxtaposition of AI in a fashion-related context with the unexpected dinosaur costume imagery, and using emojis enhances the playful tone. It's succinct, keeping within character limits, and its playful yet relatable analogy has decent potential for virality, especially among tech-savvy audiences. The hashtags add context and broaden its potential reach without being overbearing.",
 'iteration': 1,
 'max_iteration': 5,
 'tweet_history': ["OpenAI is like that friend who’s alw

In [97]:
for tweet in result['tweet_history']:
    print(tweet)

OpenAI is like that friend who’s always reading the room but still decides to show up in a dinosaur costume. Sure, it's impressive, but you gotta wonder if they really know we're just here for pizza. 🦖🍕 #AIInFashion #WhoInvitedThisBot
